In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [3]:
#### RANDOM FUNCTIONS
rng = np.random.RandomState(0)
rng.randint(0, 10, 6)


array([5, 0, 3, 3, 7, 9])

In [4]:
##### 
##summary statistics
#df.describe() 

#drop na values
df.dropna()

#fillna using another

NameError: name 'df' is not defined

## 03.08 Aggregation and Grouping

- Groupby: split - apply - combine
- Iteration over groups using groupby object
- Aggregate: more flexibility to apply a list of functions or different functions for different columns using dictionary col1:mean
- Filtering: filter data based on group properties. Keep data that returns true from a boolean function, drop data that returns false
- Transformation: transform data and return the ouput whose shape is the same as input. Eg center the data
- Apply: apply an arbitrary func to the group results. Output can e either a pd object (df/series) or a scalar

In [5]:
#### GROUPBY
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
g = df.groupby('key')
#count values in ea group
g.sum()

#select a (group of) col to apply a method
# planets.groupby('method')['orbital_period'].median()

## iterate over group
# for (method, group) in planets.groupby('method'):
#     print("{0:30s} shape={1}".format(method, group.shape))

,data
key,
A,3
B,5
C,7


In [6]:
### aggregate
##apply a list of funcs to a groupby object
df.groupby('key')['data1'].aggregate(['min', np.median, max])

##Another useful pattern is to pass a dictionary 
##mapping column names to operations to be applied on that column:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

KeyError: 'Column not found: data1'

In [51]:
#### time methods
big_array = np.random.rand(1000000)
%timeit min(big_array)
%timeit np.min(big_array)

40.2 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
457 µs ± 33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
### FILTER: drop data based on some group properties
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df
def filter_func(x):
    ## return true if data2 values have std >4
    return x['data2'].std() > 4
## drop data whose col data2 values have std < 4
# df.groupby('key').filter(filter_func)
display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

In [8]:
### TRANFORM : output same shape as input
# remove a group's mean from ea value in a group
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [9]:
### APPLY
## takes in a df, returns either a pd object (df/s) or a scalar

def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


In [11]:
#fill nan of a column using non-na values from another column
df1 = pd.DataFrame({'a':range(4), 'b': [10,np.nan, None, 5]})
print(df1)
df1['b'].fillna(df1['a'])
# df1

   a     b
0  0  10.0
1  1   NaN
2  2   NaN
3  3   5.0


0    10.0
1     1.0
2     2.0
3     5.0
Name: b, dtype: float64

## Replace. Update

- Replace: use data from  dict of key-value pairs to replace values of corresponding keys in a df
- Update: use index and matching col names to update a df

In [22]:
#### GOOD!
#### replace
# A.replace(B): use values from B (could be a dict or a df) to replace matching values from df A, based on matching index
A = pd.DataFrame(
    {'C1': ['a', 'd', 'a', 'b'],
     'C2': ['b', 'a', 'c', 'e'],
     'C3': ['c', 'e', '', 'e'],
     'C4': ['a', 'b', '', ''],
     'C5': ['', 'a', '', '']})
B = pd.DataFrame({'Code': ['a', 'b', 'c', 'd', 'e'],
                  'Value': ["'House'", "'Bike'", "'Lamp'", "'Window'", "'Car'"]})

# print(A, B)
display('A', 'B', "A.replace(B.set_index('Code')['Value'])")
# print(A.replace(B.set_index('Code')['Value']))

A
  C1 C2 C3 C4 C5
0  a  b  c  a   
1  d  a  e  b  a
2  a  c         
3  b  e  e      

B
  Code     Value
0    a   'House'
1    b    'Bike'
2    c    'Lamp'
3    d  'Window'
4    e     'Car'

A.replace(B.set_index('Code')['Value'])
         C1       C2      C3       C4       C5
0   'House'   'Bike'  'Lamp'  'House'         
1  'Window'  'House'   'Car'   'Bike'  'House'
2   'House'   'Lamp'                          
3    'Bike'    'Car'   'Car'

In [31]:
##### update
# A.update(B)
# usage: update values in A with values in B based on matching index and column names
display('A', 'C')
C = pd.DataFrame({'C2':[1, 2, 3, 5]})
A.update(C)



- Setting values, view vs copy warning
    - best answer, v. illuminating https://stackoverflow.com/a/56603453/16149514  have to check upstream is df is a view or a copy.
    - use.loc
    - below might be wrong@!
    - note that the current df variable may actually be a view, not the original df! That's why using .loc on this view will still set off warnings!
        - ex: `df2 = df1['week'1]`. `df2['week_1'] = 1` will give warnings b/c df2 is a view of df1 not a copy.
    - when using a mask on a col of a df: do like this `df.loc[(df['A'] == 'blue') & (df['B'] == 'red') & (df['C'] == 'square'),'D'] = 'M5'`
        https://stackoverflow.com/questions/21263020/pandas-update-value-if-condition-in-3-columns-are-met
- READ THIS  https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
- https://stackoverflow.com/questions/24036911/how-to-update-values-in-a-specific-row-in-a-python-pandas-dataframe
- multilevel index update https://stackoverflow.com/questions/28002197/pandas-proper-way-to-set-values-based-on-condition-for-subset-of-multiindex-da
- https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
- `df[df['C'] > 0]`

**replace vs update**
- update: use df directly. df1.update(df2). values in df1 that match index and col will be updated.
- https://stackoverflow.com/questions/27060098/replacing-few-values-in-a-pandas-dataframe-column-with-another-value



python & numpy tutorial
- 
====
READ THIS b4 posting on stackoverflow
https://stackoverflow.com/questions/20109391/how-to-make-good-reproducible-pandas-examples

https://github.com/Coldsp33d/stackoverflow-pandas-canonicals
this one looks good https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas/53954986#53954986
class bool
logial and, or, not, xor
-xor: true!= false


- rename column
- df.rename(columns={"A": "a", "B": "c"})


- add new cols to df
-add new rows to df

-----
replace, update, map
- map a df/col to a dictionary to replace values in said df/col
- call replace on df with arg the dictionary
- map faster than replace
https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict

-replace
https://stackoverflow.com/questions/37686052/mapping-values-from-one-dataframe-to-another
-update here
https://stackoverflow.com/questions/49928463/python-pandas-update-a-dataframe-value-from-another-dataframe

-----
MERGE
https://pandas.pydata.org/docs/user_guide/merging.html
cs95 https://stackoverflow.com/questions/53645882/pandas-merging-101/53645883#53645883

---
CONCAT
https://pandas.pydata.org/docs/reference/api/pandas.concat.html



-----
add new data to a DF by row/col
-soln: best way is to append all available to a python LIST then create a df
- slow:concat, append.
https://stackoverflow.com/questions/10715965/create-a-pandas-dataframe-by-appending-one-row-at-a-time?rq=1
	see shikharDua https://stackoverflow.com/a/17496530/16149514, Mikhail same and cs95
	,cs95 and others


----
rows iteration
READ THIS https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas?rq=1



----
map
https://stackoverflow.com/questions/29794959/pandas-add-new-column-to-dataframe-from-dictionary

------


stack 2 dataframes with the same shape on top of each other. But they diff col names
soln: change col names so that they have have the same col names. Then append or concat
https://stackoverflow.com/questions/45590866/python-pandas-concat-dataframes-with-different-columns-ignoring-column-names

-----
create a new columns whose values are based on some conditions
-use list comprehension
- np.where-np.select
- map
https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-columns

- nested where
https://stackoverflow.com/questions/49228596/pandas-case-when-default-in-pandas
- use pd.cut to bin and labels buckets
- np.select get  another vote
https://stackoverflow.com/questions/39109045/numpy-where-with-multiple-conditions


---
drop a row/col
df.drop(['col1', 'col2'], axis=0)

---
APPLY is a loop over rows so is very slow. use vectorized operations whenever possible
use apply with multiple args
https://stackoverflow.com/questions/49237663/passing-a-function-with-multiple-arguments-to-dataframe-apply

use apply on multiple columns:
create a lambda func that takes in a df then pass that df to our true func
https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

discussion on apply
https://stackoverflow.com/questions/54432583/when-should-i-not-want-to-use-pandas-apply-in-my-code

----

In [10]:
#create a Series from a list
L = [0.25, 0.5, 0.75, 1.0]
#2: a Series' 2 main attributes are ....
#3: indexing: item index 1
#4 slicing: values from 0 to 2
#5 create a Series with values[0.25, 0.5, 0.75, 1.0] and index ['a', 'b', 'c', 'd']

#6 create a Series from a dictionary
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
#7 show values for slice from Cali to Florida

#8 create a Series with scalar value 1 and index [1,2,3]

#9 create a Series with dict {2:'a', 1:'b', 3:'c'} and index only 2 and 3

In [14]:
s3 = pd.Series(1, index=range(1, 4))
s3.index

RangeIndex(start=1, stop=4, step=1)

In [15]:
#1
#2 

#3

#4

#5

#6

#7

#8

#9



In [10]:
#1
s = pd.Series(L)
#2 : values and index
s.values
s.index
#3
L[1]
#4
L[0:3]
#5
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=['a', 'b', 'c', 'd'])
#6
s = pd.Series(population_dict)
s['California':'Florida']

#8
pd.Series(1, index=[1,2,3])

#9
pd.Series({2:'a', 1:'b', 3:'c'}, index=[3, 2])

1    1
2    1
3    1
dtype: int64

In [17]:
#1given 2 series, create a df
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
pop = pd.Series(population_dict)
area = pd.Series(area_dict)

#2 create a dictionary with ea. dict a row

#3 access the above df's main attributes: column names, index, values, shape, len

#4 when using a list of dicts, each dict represents a ....?
data = [{'a': i, 'b': 2 * i}
        for i in range(3)]

#5[just a note] df can be created with values, columns and index

In [29]:
df = pd.DataFrame({'pop': pop, 'area':area})
df
df2 = pd.DataFrame(data)
df2
pd.DataFrame([population_dict, area_dict])

,California,Texas,New York,Florida,Illinois
0,38332521,26448193,19651127,19552860,12882135
1,423967,695662,141297,170312,149995


In [25]:
#1

#2

#3

#4 ... a row. 'a', 'b' would be the columns, and there are 3 rows


2

In [6]:
#1 data is a list of dicts: ea dict/series = a row
df0 = pd.DataFrame([population_dict, area_dict])
df1 = pd.DataFrame({'pop': pop, 'area':area})

#2 data is a dict: ea key-value pair is a column matched to its values
pd.DataFrame([population_dict, area_dict], index=['pop', 'area'])

#3 list of dicts: ea dict a row
df2 = pd.DataFrame(data)

#2
df.columns
df.values
df.index
df.shape
df.size

,California,Texas,New York,Florida,Illinois
pop,38332521,26448193,19651127,19552860,12882135
area,423967,695662,141297,170312,149995


In [14]:
#1 
states = pd.DataFrame({'population': pop,
                       'area': area})
#2
pd.DataFrame([population_dict, area_dict], index=['pop', 'area']) # when data is list of dicts, each dict will be a row
#3
df.columns
df.index
df.values
df.size
df.shape
df.dtypes
#3 a row's values
pd.DataFrame([{'a':1, 'b':2}])

#4
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

,a,b
0,1,2


In [ ]:
#1 what are the 2 main methods to access data in a df?

#.loc(name-based) and .iloc(integer-based)


In [26]:
# 03.08-Aggregation-and-Grouping-Copy1

#1 how to count; select 1st and last item; ...
#... calc mean, median; min max std dev, var, MAD, product, sum

#2 explain how groupby works
# split-apply-combine: df is split based on values of some columns. 
#then we apply some functions on each group. The results are then merged into an output array

#3 group by key and sum the data

df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

#df.groupby('key').sum()

#4 what are the most important operations for Groupby?
#aggregate, filter, transform, apply

#groupby 'method' and view 'orbital_period'
planets.groupby('method')['orbital_period'].median()

#5 iteration over groups
for (i, Y) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))
    
    
#6 use aggregate to compute (min median, max) at once 
# df.groupby('key').aggregate(['min', np.median, max])

#compute min on col data1 and max on data2
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

#filtering: what does it do?
#keep data based on same criteria. Otherwise, drop.
def filter_func(x):
    return x['data2'].std() > 4

# display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")
df.groupby('key').std()
#only groups that have std > 4 are kept
df.groupby('key').filter(filter_func)

#transformation: what does it do 
#return a df with the same shape as the group it was applied to

#apply:
#more flexible: can return either a PD object e.g df/series or a scalar
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

NameError: name 'planets' is not defined

In [ ]:
#1
# count() 	Total number of items
# first(), last() 	First and last item
# mean(), median() 	Mean and median
# min(), max() 	Minimum and maximum
# std(), var() 	Standard deviation and variance
# mad() 	Mean absolute deviation
# prod() 	Product of all items
# sum() 	Sum of all items

In [ ]:
#fill nan of a column using non-na values from another column
df1 = pd.DataFrame({'a':range(4), 'b': [10,np.nan, None, 5]})
df1['b'].fillna(df1['a'])
df1